In [54]:
import pandas as pd
import plotly.express as px
import re
import datetime
import numpy as np

from Project.Database import Db

# Example data for the TPM

In [55]:
# pd.read_csv("/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/Data/ISID.csv")

In [56]:
# pd.read_json(
#     path_or_buf="/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/output/Experiment_minsup0.1_minconf_0.1/level1.json",
#     orient='name_node')

In [57]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)

In [58]:
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self.index.str.contains("Child")) &
        (~self.index.str.contains("Prnt")) &
        (~self.index.str.contains("StatusLatentload"))
))

appliances_statusOnOff = meta.loc[appliance_condition].index.tolist()

In [59]:
#Replaces values to be binary 0 or 1.
for i in appliances_statusOnOff:
    year1_minutes.loc[year1_minutes[i] < 0, i] = 0
    year1_minutes.loc[year1_minutes[i] > 0, i] = 1

In [60]:
minute_appliances_status = year1_minutes[["Timestamp"] + appliances_statusOnOff].copy()

In [61]:
for col in meta["Measurement_Location"].unique():
    appliance_condition = (lambda self: (
            (self["Parameter"] == "Status_OnOff") &
            (self.index.str.contains("Light")) &
            (self["Measurement_Location"] == col)
    ))
    light_cols = meta.loc[appliance_condition].index
    if len(light_cols) > 1:
        minute_appliances_status[col + " Lights"] = minute_appliances_status[light_cols].max(1)
        minute_appliances_status.drop(light_cols, axis= 1, inplace=True)
        for attribute in light_cols:
            appliances_statusOnOff.remove(attribute)
        appliances_statusOnOff.append(col + " Lights")

In [62]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
minute_appliances_status["Timestamp"] = pd.to_datetime(minute_appliances_status["Timestamp"],
                                                       format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(
    unit="h", arg=(
        minute_appliances_status["Timestamp"].astype("str").str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[
            1]).astype(
        int))

#Extracts hour of Timestamp.
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear

In [63]:
# minute_appliances_status['Timestamp'] = (pd.to_datetime(minute_appliances_status["Timestamp"],
#                                                        format="%Y-%m-%d %H:%M:%S%z", utc=True).astype("int") / 1e9).astype("int")

In [64]:
minute_appliances_status['Timestamp'] = (pd.DatetimeIndex ( minute_appliances_status["Timestamp"] ).astype ( np.int64 )/1000000).astype(int)


In [65]:
minute_appliances_status["DayOfYear"] = ((minute_appliances_status['Timestamp'] - minute_appliances_status['Timestamp'].min()) / (3600 * 24)).astype("int")

In [66]:
minute_appliances_status

,Timestamp,Load_StatusApplianceCooktop,Load_StatusApplianceDishwasher,Load_StatusApplianceOven,Load_StatusApplianceRangeHood,Load_StatusBA1Lights,Load_StatusBA2Lights,Load_StatusBR2Lights,Load_StatusBR3Lights,Load_StatusBR4Lights,...,Load_StatusPlugLoadMBRTV,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadVacuum,Load_StatusPlugLoadVideoGame,Kitchen Lights,Living Room Lights,Master Bedroom Lights,DayOfYear
0,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
518788,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
518789,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
518790,-2147483648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [67]:
appliance_job_list = []

for appliance in appliances_statusOnOff:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load_Status")[-1]))
    appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", "DayOfYear", appliance]][1:]

    df_indices = appliance_switch.index.tolist()
    for index, df_index in enumerate(df_indices):
        if appliance_switch.loc[df_index, appliance]:
            try:
                appliance_job_list.append({"start": appliance_switch.loc[df_index, "Timestamp"],
                                           "end": appliance_switch.loc[df_indices[index + 1], "Timestamp"],
                                           "appliance": name,
                                           "day": appliance_switch.loc[df_indices[index], "DayOfYear"]
                                           })
            except:
                continue

appliance_job_list

[{'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'appliance': 'Appliance Cooktop',
  'day': 0},
 {'start': -2147483648,
  'end': -2147483648,
  'a

In [68]:
df = pd.DataFrame(appliance_job_list).sort_values(by=['day', 'start']).reset_index(drop=True)
df

,start,end,appliance,day
0,-2147483648,-2147483648,Appliance Cooktop,0
1,-2147483648,-2147483648,Appliance Cooktop,0
2,-2147483648,-2147483648,Appliance Cooktop,0
3,-2147483648,-2147483648,Appliance Cooktop,0
4,-2147483648,-2147483648,Appliance Cooktop,0
...,...,...,...,...
18641,-2147483648,-2147483648,Master Bedroom Lights,0
18642,-2147483648,-2147483648,Master Bedroom Lights,0
18643,-2147483648,-2147483648,Master Bedroom Lights,0
18644,-2147483648,-2147483648,Master Bedroom Lights,0


In [69]:
df.to_csv(path_or_buf=Db.get_project_path("Project/TPM/TPM/Data/houseData.csv", header=False, index=False))

In [70]:
# level2=pd.read_json(path_or_buf="/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/output/Experiment_minsup0.1_minconf_0.1/level2.json")

ValueError: Expected object or value

In [ ]:
# level2.loc[level2["name_node"] == "Entry Hall Lights,Plug Load L R Blue Ray", "patterns"].values[0][0]["conf"]